# Inverted Index construction with MapReduce

This code will firstly create a new Python file named <code>invertedIndex.py</code>. Then, it will write the following Python code into this file. The Python code of <code>invertedIndex.py</code> will be submitted to MapReduce.

The following <code>Indexer</code> class inherits from the <code>MRJob</code> class. Therefore, it will run as a MapReduce job.

Here we can overload (override) the <code>mapper</code>, <code>reducer</code>, and <code>combiner</code> methods and replace them by our code. More functions can be overloaded from the base class. A full list of the supported methods can be found <a href="https://mrjob.readthedocs.io/en/latest/job.html" target="_blank">here</a>.



## 1. Document-level inverted file

An inverted index is a data structure that for each term $t$ in the collection, it stores a (sorted) list of all document ids (postings) containing $t$.

<hr width="100%">

<img src="img/doclevel.png" style="width:60%">

<hr width="100%">

with this structure, a search engine can employ many algorithms to answer a user query. One of these algorithms is called Term-At-A-Time (TAAT):

<hr width="100%">

<img src="img/taat.png" style="width:60%">

<hr width="100%">

### 1.1. Python code for MapReduce

In [1]:
%%file python/invertedIndexDL.py
#!/usr/bin/env python3

from mrjob.job import MRJob
import string 

class Indexer(MRJob):
    # map function
    # Argument 1: self: the class itself (this)
    # Argument 2: Input key to the map function (here:none)
    # Argument 3: Input value to the map function (here:one line from the input file)
    def mapper(self, _, line):
        columns = line.replace('"', '').split(",")

        # The first column represents Document ID
        DocumentID = int(columns[0])

        # The fourth column represents the titles of the posts
        title = columns[3]

        # Perform punctuation removal
        title = title.translate(str.maketrans('', '', string.punctuation))

        # Perform casefolding
        title = title.casefold()

        # Extract the words from the title
        words = title.split()

        # Setup a dictionary for the terms of the document. The dictionaries do not contain duplicate keys.
        # So we are sure that each word is emitted one time for each document.
        doc_words = {}

        for word in words:
            if word not in doc_words:
                doc_words[word] = word

        for word in doc_words:
            # Emit a (word, Document ID) pair
            yield (word, DocumentID)

    # reduce function
    # Argument 1: self: the class itself (this)
    # Argument 2: Input key to the reduce function (here: the key that was emitted by the mapper)
    # Argument 3: Input value to the reduce function (here: a generator object; something like a
    # sorted list of ALL values associated with the same key)
    def reducer(self, word, docIDs):
        inverted_list = [ docID for docID in docIDs ]
        inverted_list.sort()
        
        yield(word, (len(inverted_list), inverted_list))

if __name__ == '__main__':
    Indexer.run()


Overwriting python/invertedIndexDL.py


### 1.2. Running in standalone mode


In [2]:
!python3 python/invertedIndexDL.py data/posts.csv -o out/localout_iiDL


Using configs in /home/bdccuser/.mrjob.conf
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/invertedIndexDL.bdccuser.20230425.085310.028785
job output is in out/localout_iiDL
Removing temp directory /tmp/invertedIndexDL.bdccuser.20230425.085310.028785...


### 1.3. Running in the Hadoop cluster in a fully/pseudo distributed mode

In [3]:
!hdfs dfs -rm -r out_iiDL
!python3 python/invertedIndexDL.py -r hadoop data/posts.csv -o out_iiDL22


rm: `out_iiDL': No such file or directory
Using configs in /home/bdccuser/.mrjob.conf
No configs specified for hadoop runner
Looking for hadoop binary in /home/hdoop/hadoop-3.2.1/bin...
Found hadoop binary: /home/hdoop/hadoop-3.2.1/bin/hadoop
Using Hadoop version 3.2.1
Looking for Hadoop streaming jar in /home/hdoop/hadoop-3.2.1...
Found Hadoop streaming jar: /home/hdoop/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar
Creating temp directory /tmp/invertedIndexDL.bdccuser.20230425.085313.716021
uploading working dir files to hdfs:///user/bdccuser/tmp/mrjob/invertedIndexDL.bdccuser.20230425.085313.716021/files/wd...
Copying other local files to hdfs:///user/bdccuser/tmp/mrjob/invertedIndexDL.bdccuser.20230425.085313.716021/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar2728416951108793872/] [] /tmp/streamjob840659835583255781.jar tmpDir=null
  Connecting to ResourceManager at /127.0.0.1:8032
  Connecting to ResourceManager at /127.0.0.1:8032
  Error Launc

### 1.4. Copy the output file from HDFS to the local file system

In [4]:
!hdfs dfs -copyToLocal -f out_iiDL22 /home/bdccuser/notebooks/mapreduce/out/out_iiDL22


2023-04-25 11:53:34,201 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


## 2. Term-level inverted file (with frequencies)

A document-level inverted index can only answer boolean queries, similarly to a database. More specifically, it can return all documents that satisfy a particular query, but it tells us nothing about the quality of the documents, or their relevance to the user query.

To overcome this problem, a term-level inverted file also contains the frequency of each word in a document (known as term-document frequency). This value along with some other statistics, provides us with the ability to perform ranked retrieval. This is a procedure where we are not particularly interested in retrieving <b>all</b> the candidate documents, but only the <b>best</b>-$k$ among them.


<hr width="100%">

<img src="img/termlevel.png" style="width:60%">

### 2.1. Python code for MapReduce

In [5]:
%%file python/invertedIndexTL.py
#!/usr/bin/env python3

from mrjob.job import MRJob
import string 

class Indexer(MRJob):
    # map function
    # Argumnet 1: self: the class itself (this)
    # Argumnet 2: Input key to the map function (here:none)
    # Argumnet 3: Input value to the map function (here:one line from the input file)
    def mapper(self, _, line):
        columns = line.replace('"', '').split(",")

        # The first column represents Document ID
        DocumentID = int(columns[0])

        # The fourth column represents the titles of the posts
        title = columns[3]

        # Perform punctuation removal
        title = title.translate(str.maketrans('', '', string.punctuation))

        # Perform casefolding
        title = title.casefold()

        # Extract the words from the title
        words = title.split()

        # Setup a dictionary for the terms of the document. The dictionaries do not contain duplicate keys.
        # So we are sure that each word is emitted one time for each document.
        doc_words = {}

        for word in words:
            if word in doc_words:
                doc_words[word] = doc_words[word] + 1
            else:
                doc_words[word] = 1


        for word in doc_words:
            # Emit a (word, Document ID) pair
            yield (word, (DocumentID, doc_words[word]))


    # reduce function
    # Argumnet 1: self: the class itself (this)
    # Argumnet 2: Input key to the reduce function (here: the key that was emitted by the mapper)
    # Argumnet 3: Input value to the reduce function (here: a generator object; something like a
    # sorted list of ALL values associated with the same key)
    def reducer(self, word, postings):
        inv_list = [ posting for posting in postings ]

        # Here the inv_list does not contain duplicates
        yield(word, (len(inv_list), inv_list))
  
if __name__ == '__main__':
    Indexer.run()


Overwriting python/invertedIndexTL.py


The above procedure constructs a document-level inverted index with MapReduce on the titles of some blog posts. The procedure is as follows:

<ol>
    <li>Map over all documents</li>
    <li>Emit the word as a key and the document ID as the value</li>
    <li>Sort/shuffle: group postings by term (automatic)</li>
    <li>Reduce: Gather and sort the postings (e.g., by docid)</li>
    <li>Write postings to disk</li>
    <li>MapReduce does all the heavy lifting!</li>
    </ol>

### 2.2. Running in standalone mode

In [6]:
!python3 python/invertedIndexTL.py data/posts.csv -o out/localout_iiTL


Using configs in /home/bdccuser/.mrjob.conf
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/invertedIndexTL.bdccuser.20230425.085334.834225
job output is in out/localout_iiTL
Removing temp directory /tmp/invertedIndexTL.bdccuser.20230425.085334.834225...


### 2.3. Running in the Hadoop cluster in a fully/pseudo distributed mode

In [7]:
!hdfs dfs -rm -r out_iiTL
!python3 python/invertedIndexTL.py -r hadoop data/posts.csv -o out_iiTL


Deleted out_iiTL
Using configs in /home/bdccuser/.mrjob.conf
No configs specified for hadoop runner
Looking for hadoop binary in /home/hdoop/hadoop-3.2.1/bin...
Found hadoop binary: /home/hdoop/hadoop-3.2.1/bin/hadoop
Using Hadoop version 3.2.1
Looking for Hadoop streaming jar in /home/hdoop/hadoop-3.2.1...
Found Hadoop streaming jar: /home/hdoop/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar
Creating temp directory /tmp/invertedIndexTL.bdccuser.20230425.085337.728660
uploading working dir files to hdfs:///user/bdccuser/tmp/mrjob/invertedIndexTL.bdccuser.20230425.085337.728660/files/wd...
Copying other local files to hdfs:///user/bdccuser/tmp/mrjob/invertedIndexTL.bdccuser.20230425.085337.728660/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar4389966928610581573/] [] /tmp/streamjob1521693950853296767.jar tmpDir=null
  Connecting to ResourceManager at /127.0.0.1:8032
  Connecting to ResourceManager at /127.0.0.1:8032
  Disabling Erasure Coding for path: 

### 2.4. Copy the output file from HDFS to the local file system

In [8]:
!hdfs dfs -copyToLocal -f out_iiTL /home/bdccuser/notebooks/mapreduce/out/out_iiTL


2023-04-25 11:54:22,905 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


## 3. Positional Indexes

A positional index allows the introduction of more sophisitcated ranking models and broadens the types of queries that can be answered by a system. For example, the inclusion of the position of a term in a document allows the processing of "exact phrase" queries. It also supports ranking models that take into consideration the notion of term-proximity to identify the best documents.

<hr width="100%">

<img src="img/positional.png" style="width:60%">

<hr width="100%">

### 3.1. Python code for MapReduce

In [9]:
%%file python/invertedIndexPI.py
#!/usr/bin/env python3

from mrjob.job import MRJob
import string 

class Indexer(MRJob):
    # map function
    # Argumnet 1: self: the class itself (this)
    # Argumnet 2: Input key to the map function (here:none)
    # Argumnet 3: Input value to the map function (here:one line from the input file)
    def mapper(self, _, line):
        columns = line.replace('"', '').split(",")

        # The first column represents Document ID
        DocumentID = int(columns[0])

        # The fourth column represents the titles of the posts
        title = columns[3]

        # Perform punctuation removal
        title = title.translate(str.maketrans('', '', string.punctuation))

        # Perform casefolding
        title = title.casefold()

        # Extract the words from the title
        words = title.split()

        # Setup a dictionary for the terms of the document
        doc_words = {}
        position = 0

        for word in words:
            position = position + 1

            if word in doc_words:
                doc_words[word]['freq'] = doc_words[word]['freq'] + 1
                doc_words[word]['pos'].append(position)

            else:
                doc_words[word] = {'freq': 1, 'pos': [position] }


        for word in doc_words:
            # Emit a (word, Document ID) pair
            yield (word, (DocumentID, doc_words[word]['freq'], doc_words[word]['pos']))


    # reduce function
    # Argumnet 1: self: the class itself (this)
    # Argumnet 2: Input key to the reduce function (here: the key that was emitted by the mapper)
    # Argumnet 3: Input value to the reduce function (here: a generator object; something like a
    # sorted list of ALL values associated with the same key)
    def reducer(self, word, postings):
        inv_list = [ posting for posting in postings ]
        yield(word, (len(inv_list), inv_list))
  
if __name__ == '__main__':
    Indexer.run()


Overwriting python/invertedIndexPI.py


### 3.2. Running in standalone mode

In [10]:
!python3 python/invertedIndexPI.py data/posts.csv -o out/localout_iiPI


Using configs in /home/bdccuser/.mrjob.conf
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/invertedIndexPI.bdccuser.20230425.085423.638200
job output is in out/localout_iiPI
Removing temp directory /tmp/invertedIndexPI.bdccuser.20230425.085423.638200...


### 3.3. Running in the Hadoop cluster in a fully/pseudo distributed mode

In [11]:
!hdfs dfs -rm -r out_iiPI
!python3 python/invertedIndexPI.py -r hadoop data/posts.csv -o out_iiPI


Deleted out_iiPI
Using configs in /home/bdccuser/.mrjob.conf
No configs specified for hadoop runner
Looking for hadoop binary in /home/hdoop/hadoop-3.2.1/bin...
Found hadoop binary: /home/hdoop/hadoop-3.2.1/bin/hadoop
Using Hadoop version 3.2.1
Looking for Hadoop streaming jar in /home/hdoop/hadoop-3.2.1...
Found Hadoop streaming jar: /home/hdoop/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar
Creating temp directory /tmp/invertedIndexPI.bdccuser.20230425.085426.509014
uploading working dir files to hdfs:///user/bdccuser/tmp/mrjob/invertedIndexPI.bdccuser.20230425.085426.509014/files/wd...
Copying other local files to hdfs:///user/bdccuser/tmp/mrjob/invertedIndexPI.bdccuser.20230425.085426.509014/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar6428007698647658147/] [] /tmp/streamjob4462153889864768540.jar tmpDir=null
  Connecting to ResourceManager at /127.0.0.1:8032
  Connecting to ResourceManager at /127.0.0.1:8032
  Disabling Erasure Coding for path: 

### 3.4. Copy the output file from HDFS to the local file system

In [12]:
!hdfs dfs -copyToLocal -f out_iiPI /home/bdccuser/notebooks/mapreduce/out/out_iiPI


2023-04-25 11:55:15,711 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
